# Export Imaging Data to Video

This notebook demonstrates how to export calcium imaging data to high-quality MP4 videos.
Videos are saved to the MBO data folder (`~/mbo/data/`).

**Use cases:**
- Show cell stability before/after registration
- Create previews for presentations or websites
- Quick visual QC of recordings

**Denoising options:**
- `temporal_smooth`: Rolling average over N frames (reduces flicker)
- `spatial_smooth`: Gaussian blur sigma (reduces pixel noise)
- Suite2p PCA denoising: Block-wise PCA for noise reduction
- [DeepCAD](https://github.com/cabooster/DeepCAD): Deep learning denoising (separate install)

In [25]:
from pathlib import Path
import numpy as np
from mbo_utilities import imread, to_video, get_mbo_dirs

# Output directory (created automatically on mbo_utilities install)
output_dir = get_mbo_dirs()["data"]
output_dir.mkdir(parents=True, exist_ok=True)
print(f"Videos will be saved to: {output_dir}")

Videos will be saved to: C:\Users\RBO\mbo\data


## Load Your Data

Replace the path below with your own data file.

In [44]:
import mbo_utilities as mbo
arr = mbo.imread(r"D:\example_extraction\suite2p")

Inconsistent frame counts across planes: [1574, 1500, 1574, 1574, 1574, 1500, 1500, 1574, 1574, 1574, 1574, 1574, 1574, 1574]. Using minimum: 1500


## Quick Preview Video (10x Speed)

Fast export to check data quality and cell stability.

In [45]:
# preview_path = output_dir / "preview_10x.mp4"

# to_video(
#     arr,
#     preview_path,
#     fps=30,              # Base frame rate
#     speed_factor=10,     # 10x faster playback
#     quality=7,           # Medium quality for quick preview
# )

# print(f"\nPreview saved to: {preview_path}")

## High-Quality Export for Website

Publication-quality video with smoothing and gamma correction.

In [51]:
hq_path = output_dir / "high_quality.mp4"

to_video(
    arr,
    hq_path,
    fps=30,
    speed_factor=2,       # 5x speed - good balance of speed and detail
    temporal_smooth=3,    # Rolling average over 3 frames (reduces flicker)
    spatial_smooth=0.5,   # Slight Gaussian blur (reduces noise)
    gamma=0.8,            # Brighten midtones (good for calcium imaging)
    vmin_percentile=2,    # Clip dark values
    vmax_percentile=99,   # Clip bright outliers
    quality=10,           # Maximum quality
)

print(f"\nHigh-quality video saved to: {hq_path}")

Writing video:   0%|          | 0/1500 [00:00<?, ?frames/s]

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (448, 550) to (448, 560) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).



High-quality video saved to: C:\Users\RBO\mbo\data\high_quality.mp4


## With Colormap

Apply a matplotlib colormap for better visualization.

In [29]:
color_path = output_dir / "colormap_viridis.mp4"

to_video(
    arr,
    color_path,
    fps=30,
    speed_factor=5,
    temporal_smooth=3,
    gamma=0.8,
    cmap="viridis",       # Try: "hot", "magma", "inferno", "gray"
    quality=10,
    max_frames=500,
)

print(f"\nColormap video saved to: {color_path}")

Writing video:   0%|          | 0/500 [00:00<?, ?frames/s]

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (448, 550) to (448, 560) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).



Colormap video saved to: C:\Users\RBO\mbo\data\colormap_viridis.mp4


## Suite2p-style Denoising

Suite2p's detection pipeline uses multiple steps:
1. **Temporal binning** - average every N frames
2. **PCA denoising** - block-wise PCA reconstruction  
3. **Temporal high-pass filter** - subtract rolling mean (removes baseline drift)
4. **Normalize by std** - divide by temporal standard deviation

The high-pass filter is key - it subtracts the slow baseline drift to enhance transients.

In [47]:
from suite2p.detection.denoise import pca_denoise
from suite2p.detection.utils import temporal_high_pass_filter, standard_deviation_over_time
from scipy.ndimage import uniform_filter1d

# Load a subset of frames
n_frames = 300
mov = np.asarray(arr[:n_frames, 0]).astype(np.float32)  # plane 0
print(f"Original: {mov.shape}, dtype: {mov.dtype}")

# Step 1: Temporal smoothing (rolling average)
temporal_window = 5
mov_smooth = uniform_filter1d(mov, size=temporal_window, axis=0)
print(f"After temporal smooth (window={temporal_window}): {mov_smooth.shape}")

# Step 2: Temporal high-pass filter (subtract rolling mean - removes baseline drift)
high_pass_width = 50  # frames - removes slow drift, keeps fast transients
mov_hp = temporal_high_pass_filter(mov_smooth.copy(), width=high_pass_width)
print(f"After high-pass (width={high_pass_width}): range [{mov_hp.min():.1f}, {mov_hp.max():.1f}]")

# Step 3: Normalize by temporal std (enhances signal relative to noise)
sdmov = standard_deviation_over_time(mov_hp, batch_size=100)
mov_norm = mov_hp / sdmov
print(f"After std normalization: range [{mov_norm.min():.1f}, {mov_norm.max():.1f}]")

Original: (300, 550, 448), dtype: float32
After temporal smooth (window=5): (300, 550, 448)
After high-pass (width=50): range [-4070.1, 5368.8]
After std normalization: range [-12.9, 18.3]


In [48]:
# Export comparison videos
# Original with temporal smooth only
to_video(mov_smooth, output_dir / "01_temporal_smooth.mp4", fps=30, speed_factor=5, quality=10)

# After high-pass (baseline removed) - need to rescale for display
mov_hp_display = mov_hp - mov_hp.min()  # shift to positive
to_video(mov_hp_display, output_dir / "02_high_pass.mp4", fps=30, speed_factor=5, quality=10)

# After std normalization (transients enhanced)
mov_norm_display = np.clip(mov_norm, -5, 5)  # clip outliers
mov_norm_display = (mov_norm_display - mov_norm_display.min()) / (mov_norm_display.max() - mov_norm_display.min())
mov_norm_display = (mov_norm_display * 255).astype(np.uint8)
to_video(mov_norm_display, output_dir / "03_std_normalized.mp4", fps=30, speed_factor=5, quality=10)

print("Exported comparison videos")

Writing video:   0%|          | 0/300 [00:00<?, ?frames/s]

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (448, 550) to (448, 560) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Writing video:   0%|          | 0/300 [00:00<?, ?frames/s]

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (448, 550) to (448, 560) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Writing video:   0%|          | 0/300 [00:00<?, ?frames/s]

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (448, 550) to (448, 560) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Exported comparison videos


In [49]:
# Export denoised video
denoised_path = output_dir / "pca_denoised.mp4"

to_video(
    mov_denoised,
    denoised_path,
    fps=30,
    speed_factor=5,
    gamma=0.8,
    quality=10,
)

print(f"\nPCA denoised video saved to: {denoised_path}")

Writing video:   0%|          | 0/200 [00:00<?, ?frames/s]

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (448, 550) to (448, 560) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).



PCA denoised video saved to: C:\Users\RBO\mbo\data\pca_denoised.mp4


## Export from 4D Data (Multi-plane)

For volumetric data, select which z-plane to export.

In [50]:
# Create synthetic 4D data (T, Z, Y, X)

# Export plane 2 (0-indexed)
plane_path = output_dir / "plane_2.mp4"

to_video(
    arr,
    plane_path,
    fps=30,
    speed_factor=5,
    plane=2,              # Export z-plane 2 (0-indexed)
    temporal_smooth=3,
    quality=9,
)

print(f"\nPlane video saved to: {plane_path}")

Writing video:   0%|          | 0/1500 [00:00<?, ?frames/s]

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (448, 550) to (448, 560) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).



Plane video saved to: C:\Users\RBO\mbo\data\plane_2.mp4


## Export First N Frames Only

Useful for quick tests or creating short clips.

In [34]:
clip_path = output_dir / "first_100_frames.mp4"

to_video(
    arr,
    clip_path,
    fps=30,
    speed_factor=2,
    max_frames=100,       # Only export first 100 frames
    quality=9,
)

print(f"\nClip saved to: {clip_path}")

Writing video:   0%|          | 0/100 [00:00<?, ?frames/s]

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (448, 550) to (448, 560) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).



Clip saved to: C:\Users\RBO\mbo\data\first_100_frames.mp4


## Summary

List all exported videos.

## Denoising References

**Built-in options:**
- `temporal_smooth`: Rolling average - fast, good for reducing frame-to-frame flicker
- `spatial_smooth`: Gaussian blur - reduces pixel noise but can blur fine details
- Suite2p `pca_denoise`: Block-wise PCA reconstruction - good balance of speed and quality

**External tools (require separate installation):**
- [DeepCAD](https://github.com/cabooster/DeepCAD): Deep self-supervised learning denoising (`pip install deepcad`)
- [DeepCAD-RT](https://github.com/cabooster/DeepCAD-RT): Real-time version of DeepCAD
- [DeepInterpolation](https://github.com/AllenInstitute/deepinterpolation): Allen Institute's deep learning denoiser

**References:**
- Li et al. (2021) "Reinforcing neuron extraction and spike inference in calcium imaging using deep self-supervised denoising" *Nature Methods*
- Lecoq et al. (2021) "Removing independent noise in systems neuroscience data using DeepInterpolation" *Nature Methods*

In [35]:
print("Exported videos:")
print("=" * 60)
for mp4 in sorted(output_dir.glob("*.mp4")):
    size_mb = mp4.stat().st_size / 1024 / 1024
    print(f"  {mp4.name:30} {size_mb:6.2f} MB")
print("=" * 60)
print(f"\nAll videos saved to: {output_dir}")

Exported videos:
  colormap_viridis.mp4             9.88 MB
  first_100_frames.mp4             7.12 MB
  high_quality.mp4                36.85 MB
  pca_denoised.mp4                10.72 MB
  plane_2.mp4                     36.10 MB
  preview_10x.mp4                 83.47 MB

All videos saved to: C:\Users\RBO\mbo\data
